In [1]:
"""Logistic Regression Project: Used to predict the liklihood of existing customers churning, and validating the model on new customers to advise higher customer
service attention to the customers at a higher risk of churning"""

In [2]:
#Load PySpark
import pyspark

In [3]:
#Load and explore data
df = sqlContext.sql("SELECT * FROM customer_churn")
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [4]:
#Explore data
[print(item) for item in df.head(1)[0]]

Cameron Williams
42.0
11066.8
0
7.22
8.0
2013-08-30 07:00:40
10265 Elizabeth Mission Barkerburgh, AK 89518
Harvey LLC
1
 Out[ 32 ]: [None, None, None, None, None, None, None, None, None, None]

In [5]:
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [6]:
#Check for missing data
[df.where(df[column].isNull()).count() for column in df.columns]

Out[ 34 ]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [7]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+

In [8]:
print((df.count(), len(df.columns)))

(900, 10)

In [9]:
from pyspark.ml.feature import VectorAssembler

In [10]:
df.columns

Out[ 38 ]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [11]:
#Assemble features
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'], outputCol='features')

In [12]:
output = assembler.transform(df)
output.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn| features|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|[42.0,11066.8,0.0...|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|[41.0,11916.22,0....|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|[38.0,12884.75,0....|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|[42.0,8010.76,0.0...|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|[37.0,9191.58,0.0...|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|[48.0,10356.02,0....|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|[44.0,11331.58,1....|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|[32.0,9885.12,1.0...|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|[43.0,14062.6,1.0...|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|[40.0,8066.94,1.0...|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|[30.0,11575.37,1....|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|[45.0,8771.02,1.0...|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|[45.0,8988.67,1.0...|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|[40.0,8283.32,1.0...|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|[41.0,6569.87,1.0...|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|[38.0,10494.82,1....|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|[45.0,8213.41,1.0...|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|[43.0,11226.88,0....|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|[53.0,5515.09,0.0...|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|[46.0,8046.4,1.0,...|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+--------------------+
only showing top 20 rows

In [13]:
#Create modeling dataframe
data = output.select(['features','churn'])

In [14]:
data.show()

+--------------------+-----+
 features|churn|
+--------------------+-----+
[42.0,11066.8,0.0...| 1|
[41.0,11916.22,0....| 1|
[38.0,12884.75,0....| 1|
[42.0,8010.76,0.0...| 1|
[37.0,9191.58,0.0...| 1|
[48.0,10356.02,0....| 1|
[44.0,11331.58,1....| 1|
[32.0,9885.12,1.0...| 1|
[43.0,14062.6,1.0...| 1|
[40.0,8066.94,1.0...| 1|
[30.0,11575.37,1....| 1|
[45.0,8771.02,1.0...| 1|
[45.0,8988.67,1.0...| 1|
[40.0,8283.32,1.0...| 1|
[41.0,6569.87,1.0...| 1|
[38.0,10494.82,1....| 1|
[45.0,8213.41,1.0...| 1|
[43.0,11226.88,0....| 1|
[53.0,5515.09,0.0...| 1|
[46.0,8046.4,1.0,...| 1|
+--------------------+-----+
only showing top 20 rows

In [15]:
#Split data into train, test sets
train, test = data.randomSplit([0.7,0.3])

In [16]:
from pyspark.ml.classification import LogisticRegression

In [17]:
#Initiate logistic regression
model = LogisticRegression(labelCol='churn')

In [18]:
fitted = model.fit(train)

In [19]:
summary = fitted.summary

<pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x7f59df47ec88>

In [20]:
summary.predictions.describe().show()

+-------+-------------------+-------------------+
summary| churn| prediction|
+-------+-------------------+-------------------+
 count| 619| 619|
 mean|0.18093699515347333|0.13893376413570274|
 stddev| 0.3852773063912312| 0.3461571193390968|
 min| 0.0| 0.0|
 max| 1.0| 1.0|
+-------+-------------------+-------------------+

In [21]:
 from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
#Evaluate performance of predictions and their respective probabilities (compared to actual churn)
pred = fitted.evaluate(test)

In [23]:
pred.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
 features|churn| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[22.0,11254.38,1....| 0|[4.26662201256418...|[0.98616499921414...| 0.0|
[27.0,8628.8,1.0,...| 0|[5.17207563602036...|[0.99435921697193...| 0.0|
[29.0,9378.24,0.0...| 0|[4.46410352688932...|[0.98861607205533...| 0.0|
[29.0,9617.59,0.0...| 0|[4.18268102593164...|[0.98497174746228...| 0.0|
[29.0,10203.18,1....| 0|[3.56198470098001...|[0.97240089185456...| 0.0|
[29.0,13240.01,1....| 0|[6.39606471258934...|[0.99833466909191...| 0.0|
[29.0,13255.05,1....| 0|[3.86816320346488...|[0.97953101732886...| 0.0|
[30.0,6744.87,0.0...| 0|[3.24862377529638...|[0.96262362845962...| 0.0|
[30.0,8874.83,0.0...| 0|[2.94641521304890...|[0.95009378767302...| 0.0|
[30.0,10183.98,1....| 0|[2.65383384633080...|[0.93424689474013...| 0.0|
[30.0,10744.14,1....| 1|[1.65287124513127...|[0.83927872990931...| 0.0|
[31.0,10058.87,1....| 0|[4.31329484310510...|[0.98678754533194...| 0.0|
[32.0,8617.98,1.0...| 1|[0.92990182860331...|[0.71705536822046...| 0.0|
[32.0,11715.72,0....| 0|[3.11741000109125...|[0.95760520527066...| 0.0|
[32.0,12254.75,1....| 0|[2.37342171244444...|[0.91477799345323...| 0.0|
[32.0,12479.72,0....| 0|[4.29946475590971...|[0.98660601098297...| 0.0|
[33.0,5738.82,0.0...| 0|[4.31198812009424...|[0.98677049758508...| 0.0|
[33.0,7492.9,0.0,...| 0|[4.68846189351543...|[0.99088305629283...| 0.0|
[33.0,10309.71,1....| 0|[5.93238070449214...|[0.99735485489265...| 0.0|
[33.0,12115.91,1....| 0|[2.24465443040589...|[0.90418843880142...| 0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows

In [24]:
#Initiate BinaryClassificationEvaluator to compare predictions with actual churn
churn = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='churn')

BinaryClassificationEvaluator_e623bbc03e94

In [25]:
#Evaluate with AUC
auc = churn.evaluate(pred.predictions)
print(auc)

0.7750703920294564

In [26]:
#Run model
val_model = model.fit(data)

In [27]:
#Load in new, unseen validation data
new_cust = sqlContext.sql("SELECT * FROM customers")

In [28]:
#Assemble new data features
test_new_cust = assembler.transform(new_cust)

In [29]:
test_new_cust.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: timestamp (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- features: vector (nullable = true)

In [30]:
#Transform unseen data with model
final_results = val_model.transform(test_new_cust)

In [31]:
#Final results
final_results.select('Company', 'prediction','probability').show()

+----------------+----------+--------------------+
 Company|prediction| probability|
+----------------+----------+--------------------+
 King Ltd| 0.0|[0.90218018099700...|
 Cannon-Benson| 1.0|[0.00198380445828...|
Barron-Robertson| 1.0|[0.02255110110417...|
 Sexton-Golden| 1.0|[0.00608622642083...|
 Wood LLC| 0.0|[0.75115067517455...|
 Parks-Robbins| 1.0|[0.15582819767621...|
+----------------+----------+--------------------+